In [ ]:
from pymongo import MongoClient

# Conectar ao MongoDB (considerando que está rodando no Docker na porta padrão 27017)
client = MongoClient("mongodb://localhost:27017/")

# Selecionar o banco de dados
db = client["meu_banco_de_dados"]

# Criar uma coleção (se não existir, será criada automaticamente)
colecao = db["minha_colecao"]

# Inserir documentos na coleção
documento = {
    "nome": "Produto 1",
    "preco": 100.0,
    "categoria": "Categoria A"
}

colecao.insert_one(documento)

# Verificar se o documento foi inserido
for doc in colecao.find():
    print(doc)
